# Dino Bone Finding PyTorch Model to OpenVINO for High Performant Deployment

This notebook demonstrates the steps in converting the dinosour bone finding PyTorch model to OpenVINO Intermediate Representation (IR) format.

First, the PyTorch model is converted and exported to ONNX. Then, the ONNX model is either feeded directly into OpenVINO Runtime, or the ONNX model is then convert and optimized in the OpenVINO Intermediate Representation (IR) formats. Both ONNX and IR models are executed on OpenVINO Inference Engine to show model inferencing on CPU, iGPU and/or VPU devices interchangably. 

Authors: Zhuo Wu, PhD (zhuo.wu@intel.com)  Raymond Lo, PhD (OpenVINO Edge AI Software Evangelist - Intel) 

## Imports

In [ ]:
import sys
import time
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt

from IPython.display import Markdown, display

import cv2
import numpy as np
from PIL import Image
from openvino.runtime import AsyncInferQueue, Core, InferRequest, Layout, Type
import openvino.runtime as ov

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader

## Settings

Set the name for the model. Define the path for the trained dinosaur bone finding model with PyTorch format, and the path for the converted onnx and OpenVINO IR format.

In [ ]:
DIRECTORY_NAME = "models"
BASE_MODEL_NAME = DIRECTORY_NAME + f"/resnet18-Gold20220530"
BASE_MODEL_NAME = DIRECTORY_NAME + f"/bc_resnet18_simple_NOIPEX_6Epochs_StateDict_gold"
#BASE_MODEL_NAME = DIRECTORY_NAME + f"/bc_resnet18_simple_NOIPEX_30Epochs_gold"

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
model_path = Path(BASE_MODEL_NAME).with_suffix(".pt")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

## ONNX Model Conversion

The output for this cell will show some warnings. These are most likely harmless. Conversion succeeded if the last line of the output says ONNX model exported to resnet18-Gold20220530.onnx.

In [ ]:
# scratch_model = models.resnet18(pretrained=True)
# num_ftrs = scratch_model.fc.in_features
# classes = 3
# scratch_model.fc = nn.Linear(num_ftrs, classes)
# #scratch_model.load_state_dict(torch.load('./models/resnet18-Gold20220530.pt'))
# scratch_model.load_state_dict(torch.load('./models/bc_resnet18_simple_NOIPEX_30Epochs.pt'))

In [ ]:
scratch_model = models.resnet18(pretrained=True)
num_ftrs = scratch_model.fc.in_features
classes = 3
scratch_model.fc = nn.Linear(num_ftrs, classes)
#scratch_model.load_state_dict(torch.load('./models/resnet18-Gold20220530.pt'))
scratch_model.load_state_dict(torch.load(f'./{BASE_MODEL_NAME}.pt'))


In [ ]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)

    # For the Fastseg model, setting do_constant_folding to False is required
    # for PyTorch>1.5.1
    torch.onnx.export(
        scratch_model,
        dummy_input,
        onnx_path,
        opset_version=11,
        do_constant_folding=False,
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

## Convert ONNX Model to OpenVINO IR Format

Call the OpenVINO Model Optimizer tool to convert the ONNX model to OpenVINO IR with FP16 precision. The models are saved to the current directory. We add the mean values to the model and scale the output with the standard deviation with --scale_values. With these options, it is not necessary to normalize input data before propagating it through the network.

See the [Model Optimizer Developer Guide](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer.

Executing this command may take a while. There may be some errors or warnings in the output. Model Optimization was successful if the last lines of the output include [ SUCCESS ] Generated IR version 11 model.

In [ ]:
# Construct the command for Model Optimizer
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP16
                 --output_dir "{model_path.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

In [ ]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    mo_result = %sx $mo_command
    print("\n".join(mo_result))
    if 'bad interpreter' in mo_result[0]:
        print("something went wrong")
        print("run the mo_command string from the command line after activating openvinopytorch")
        print("copy this line to command line:")
        print(mo_command)
else:
    print(f"IR model {ir_path} already exists.")


## Run Inference on Single Image with ONNX and IR Format and Show Results

Inference Engine can load ONNX models directly. We first load the ONNX model, do inference and show the results. After that we load the model that was converted to Intermediate Representation (IR) with Model Optimizer and do inference on that model and show the results on a single image.

In [ ]:
input_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size,input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size,input_size)),        
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
ImagePath = Path("assets/HFNoBone029.png")
img = Image.open(ImagePath)    
x_test = data_transforms['val'](img)[:3]   #3 channels in case png bobc
x_test.unsqueeze_(0)  # Add batch dimension
x_test2 = Variable(x_test)

### Run Inference on ONNX Format

In [ ]:
# Load network to Inference Engine
core = Core()
model_onnx = core.read_model(model=onnx_path)
compiled_model_onnx = core.compile_model(model=model_onnx, device_name="CPU")

output_layer_onnx = compiled_model_onnx.output(0)

# Run inference on the input image
res_onnx = compiled_model_onnx([x_test2])[output_layer_onnx]

In [ ]:
# print out the inference results using the onnx model format
print("inference result using onnx format for the 3 classes are ", res_onnx)

In [ ]:
def softmax(x):
    """
    Compute the softmax of vector x.
    """
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

In [ ]:
class_names = ['0', '1', '2']
predArgmax = np.argmax(res_onnx)
confidence = softmax(res_onnx)
score = []
score.append(class_names[predArgmax] )
score.append(float(confidence[0][predArgmax]) )

In [ ]:
# print out the final score, including the class name for the input map image, and its confidence level
print(score)

### Run Inference on IR Format

This will run the model on the CPU using OpenVINO Runtime

In [ ]:
# Load the network in Inference Engine
core = Core()
model_ir = core.read_model(model=ir_path)
compiled_model_ir = core.compile_model(model=model_ir, device_name="CPU")

# Get input and output layers
output_layer_ir = compiled_model_ir.output(0)

# Run inference on the input image
res_ir = compiled_model_ir([x_test2])[output_layer_ir]

In [ ]:
# get input and output nodes
input_layer = compiled_model_ir.input
output_layer = compiled_model_ir.output(0)

In [ ]:
# print out the inference results using the IR model format
print("inference result using IR format for the 3 classes are ", res_ir)

In [ ]:
predArgmax = np.argmax(res_ir)
confidence = softmax(res_ir)
score = []
score.append(class_names[predArgmax] )
score.append(float(confidence[0][predArgmax]) )

In [ ]:
# print out the final score, including the class name for the input map image, and its confidence level
print(score)

### PyTorch Score Comparison

Compare the inference results using the PyTorch model format

In [ ]:
with torch.no_grad():
    result_torch = scratch_model(torch.as_tensor(x_test2).float())

predArgmax = torch.argmax(result_torch[0]).numpy()
confidence = F.softmax(result_torch, dim=0)
score = []
score.append( class_names[predArgmax] )
score.append( float(confidence[0][predArgmax]) )

In [ ]:
# print out the final score, including the class name for the input map image, and its confidence level
print(score)

## Performance Comparison on Single Image Comparison

Measure the time it takes to perform inference on 20 images. This gives an indication of performance. For more accurate benchmarking, please use the OpenVINO Benchmark Tool. Note that many optimizations are possible to improve the performance. 
See examples: https://github.com/openvinotoolkit/openvino_notebooks/blob/main/notebooks/104-model-tools/104-model-tools.ipynb

In [ ]:
num_images = 400

input_image = x_test2

with torch.no_grad():
    start = time.perf_counter()
    for _ in range(num_images):
        scratch_model(torch.as_tensor(input_image).float())
    end = time.perf_counter()
    time_torch = end - start
print(
    f"PyTorch model on CPU: {time_torch/num_images:.3f} seconds per image, "
    f"FPS: {num_images/time_torch:.2f}"
)

start = time.perf_counter()
for _ in range(num_images):
    compiled_model_onnx([x_test2])
end = time.perf_counter()
time_onnx = end - start
print(
    f"ONNX model in Inference Engine/CPU: {time_onnx/num_images:.3f} "
    f"seconds per image, FPS: {num_images/time_onnx:.2f}"
)

start = time.perf_counter()
for _ in range(num_images):
    compiled_model_ir([input_image])
end = time.perf_counter()
time_ir = end - start
print(
    f"IR model in Inference Engine/CPU: {time_ir/num_images:.3f} "
    f"seconds per image, FPS: {num_images/time_ir:.2f}"
)

if "GPU" in core.available_devices:
    compiled_model_onnx_gpu = core.compile_model(model=model_onnx, device_name="GPU")
    start = time.perf_counter()
    for _ in range(num_images):
        compiled_model_onnx_gpu([input_image])
    end = time.perf_counter()
    time_onnx_gpu = end - start
    print(
        f"ONNX model in Inference Engine/GPU: {time_onnx_gpu/num_images:.3f} "
        f"seconds per image, FPS: {num_images/time_onnx_gpu:.2f}"
    )

    compiled_model_ir_gpu = core.compile_model(model=model_ir, device_name="GPU")
    start = time.perf_counter()
    for _ in range(num_images):
        compiled_model_ir_gpu([input_image])
    end = time.perf_counter()
    time_ir_gpu = end - start
    print(
        f"IR model in Inference Engine/GPU: {time_ir_gpu/num_images:.3f} "
        f"seconds per image, FPS: {num_images/time_ir_gpu:.2f}"
    )

## Run Inference with Images Tiling from Big Map with Single Image Inference

Best map to score Green square roughly 265 m x 265 m - about 2.5 football or soccer fields long The are in Green is a significantly smaller search are than the entire map. \
Run Inference on the whole folder of 224 maps and merge the final results into a big map. This time, image batches are first created, and then the inference will be performed on batches. Results are compared between PyTorch and OpenVINO IR formats on image batches. \
Define pre-processing functions for images tiling from a big map.

In [ ]:
def image_preprocessing(ImagePath):
    """Preprocessing for the input aerial 224*224 map image.
    """
    img = Image.open(ImagePath)    
    x_test = data_transforms['val'](img)[:3]   #3 channels in case png bobc
    x_test.unsqueeze_(0)  # Add batch dimension
    x_test2 = Variable(x_test)
    return x_test2

In [ ]:
green = Image.new('RGBA',(224,224),(0,255,0,60))
white = Image.new('RGBA',(224,224),(255,255,255,1))
lightGreen = Image.new('RGBA',(224,224),(0,255,0,20))
black = Image.new('RGBA',(224,224),(0,0,0,1))

my_classes = ['0', '1', '2']

lookup = {'0': 'No Bones',
          '1': 'Bones possible',
          '2': 'Bones highly likely'
         }

### Running single image inference on the input images with PyTorch 

In [ ]:
def scoreSingleImage_torch(ImagePath, model, dataset_classes):
    """Run inference on single image with PyTorch format.
    """
    model.eval()
    x_test2 = image_preprocessing(ImagePath)
    
    start_infer = time.perf_counter()
    output = model(x_test2)
    stop_infer = time.perf_counter()
    time_infer = stop_infer - start_infer

    class_names = dataset_classes
    predArgmax = torch.argmax(output[0]).numpy()
    confidence = F.softmax(output, dim=0)
    score = []
    score.append( class_names[predArgmax] )
    score.append( float(confidence[0][predArgmax]) )
    return score, time_infer 

In [ ]:
def result_to_map(pred, filenameMap):
    """Color each 224 * 224 map with inference results corresponded colors.
       Green for 'Bones highly likely', lightGreen for 'Bones possible', and white for 'No Bones'.
    """
    if pred == '2':
        Image.alpha_composite(img.convert("RGBA"), green).save(filenameMap)
    elif pred == '1': 
        Image.alpha_composite(img.convert("RGBA"), lightGreen).save(filenameMap)
    else:
        Image.alpha_composite(img.convert("RGBA"), black).save(filenameMap)
        
def save_merge_map(merge_path):
    """ Merge the 224 * 224 map into a large map for the final bone finding results.
    """
    map_save = merge_path
    xblock = 17
    yblock = 15
    dst = Image.new('RGB', ((xblock - 1)*224, (yblock - 1)*224))
    for x in range(xblock):
        for y in range(yblock):
            path = 'assets/224Map_auto/DNM_x{:02d}y{:02d}.png'.format(x,y)
            img = Image.open(path)
            dst.paste(img, (x*224, y*224))
            img.close()
    dst.save(map_save)
    print(map_save)
    print("Done!")

In [ ]:
import os
scratch_model = models.resnet18(pretrained=True)
num_ftrs = scratch_model.fc.in_features
classes = 3
scratch_model.fc = nn.Linear(num_ftrs, classes)
scratch_model.load_state_dict(torch.load(model_path))
score_torch = []
mapPath = 'assets/224Map'
if not os.path.exists(mapPath):
    os.makedirs(mapPath)
start_time = time.perf_counter()
infer_time_sum = 0
xblock = 17
yblock = 15
for x in range(xblock):
    for y in range(yblock):
        filename = 'assets/224/DNM_x{:02d}y{:02d}.jpg'.format(x, y)
        filenameMap = 'assets/224Map/DNM_x{:02d}y{:02d}.png'.format(x, y)
        img = Image.open(filename)
        try: 
            pred, time_infer = scoreSingleImage_torch(filename, scratch_model, my_classes)
            infer_time_sum += time_infer
            score_torch.append(pred)
            result_to_map(pred, filenameMap)
        except:
            print ("Problem", x, y, filename)

print("Scoring time elapsed for Pytorch: ", time.perf_counter() - start_time) 
print("Inferencing time elapsed for Pytorch: ", infer_time_sum) 

### Running single image inference on the input images with OpenVINO IR format 

In [ ]:
def scoreSingleImage_ir(ImagePath, compiled_model_ir, output_layer_ir, dataset_classes):
    """Run inference on single image with IR format.
    """
    x_test2 = image_preprocessing(ImagePath)
    
    start_infer = time.perf_counter()
    # Run inference on the input image
    res_ir = compiled_model_ir([x_test2])[output_layer_ir]    
    stop_infer = time.perf_counter()
    time_infer = stop_infer - start_infer
    
    predArgmax = np.argmax(res_ir)
    confidence = softmax(res_ir)
    score = []
    score.append(class_names[predArgmax] )
    score.append(float(confidence[0][predArgmax]) )
    return score, time_infer

In [ ]:
model_ir = core.read_model(model=ir_path)
config = {"PERFORMANCE_HINT": "THROUGHPUT",
          "ALLOW_AUTO_BATCHING": "Yes"}
compiled_model_ir_cpu_gpu = core.compile_model(model_ir, "GPU")
output_layer_ir = compiled_model_ir_cpu_gpu.output(0)
score_ir = []
start_time = time.perf_counter()
infer_time_sum = 0
xblock = 17
yblock = 15
mapPath = 'assets/224Map_auto'
if not os.path.exists(mapPath):
    os.makedirs(mapPath)
for x in range(xblock):
    for y in range(yblock):
        filename = 'assets/224/DNM_x{:02d}y{:02d}.jpg'.format(x, y)
        filenameMap = 'assets/224Map_auto/DNM_x{:02d}y{:02d}.png'.format(x, y)
        img = Image.open(filename)
        try: 
            pred, time_infer = scoreSingleImage_ir(filename, compiled_model_ir_cpu_gpu, output_layer_ir, my_classes)
            infer_time_sum += time_infer
            score_ir.append(pred)
            # print out the predicted class names for the input images
            # print(filename.split('/')[-1], lookup[pred[0]])
            result_to_map(pred, filenameMap)
        except:
            print ("Problem", x, y, filename)
print("Scoring time elapsed for IR: ", time.perf_counter() - start_time) 
print("Inferencing time elapsed for OpenVINO IR: ", infer_time_sum) 

### Compare the results/accuracy between PyTorch and OpenVINO

In [ ]:
counter = 0
for k in range(len(score_ir)):
    if score_torch[k][0] != score_ir[k][0]:
    #if int(score_auto[k]) != int(score_ir[k][0]):
        counter += 1
        #print('comparing between pytorch and ir')
        #print(k)
        #print(score_batch[k])
        #print(score_ir[k][0])
print('counter is', counter)

## Creat Image Batches and Run Inference with Images Tiling from Big Map with Batch Inference

In [ ]:
# File path for images tiling from a big map

import shutil

os.makedirs("data/ThreeClassManualRemove0s/test", exist_ok=True)
os.makedirs("data/ThreeClassManualRemove0s/test/unknown", exist_ok=True)
source_path = "data/DinosaurNationalMonument/20220514/224/"
target_path = "data/ThreeClassManualRemove0s/test/unknown/"
#source_path = "data/NMPanorama/224/"
#target_path = "data/NMPanorama/224/unknown/"
if not os.path.exists(target_path):
    os.makedirs(target_path)
if os.path.exists(source_path):
    shutil.rmtree(target_path)
shutil.copytree(source_path, target_path)

print('copy dir finished!')

### Run batch inference with PyTorch format

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """custom ImageFolder to get the filepaths along with the image and label data."""

    def __getitem__(self, index):
        paths = ((self.imgs[index][0]),)
        return super().__getitem__(index) + paths


def infer(model, data_path: str):
    """give trained `model` & `data_path` where images whose
    labels have to be predicted are kept.

    `data_path`: path to data eg. ./test/<random_class>/*.png
    it's important to have a folder with a`random class` name as ImgFolder
    expects it.

    returns: (
        images: images loaded from disk for inferece,
        yhats: predicted labels
        paths: image file-path on disk        )
    """
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize(*imagenet_stats)]
    )
    data = ImageFolderWithPaths(data_path, transform=transform)
    dataloader = DataLoader(
        data,
        batch_size=16,
    )
    yhats = []
    images = []
    paths = []
    
    infer_time_sum = 0
    start_time = time.perf_counter()
    for (imgs, _, fpaths) in dataloader:
        start = time.perf_counter()
        yhat = model(imgs)
        end = time.perf_counter()
        time_torch = end - start
        yhat = yhat.max(1)[1]
        yhat = yhat.data.cpu().numpy()
        yhats.extend(yhat)
        paths.extend(fpaths)
        images.extend(imgs.data.cpu())
        infer_time_sum += time_torch
    end_time = time.perf_counter()    
    print(
        f"PyTorch model on CPU with batch inference: {infer_time_sum} total time, "
    )
    print("Overall running time elapsed for PyTorch format batch inference: ", end_time - start_time)
    return images, yhats, paths

In [ ]:
imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]
images, yhats, img_paths = infer(
    scratch_model, data_path="./data/ThreeClassManualRemove0s/test"
)
#infer_dataloader = DataLoader([*zip(images, yhats)], batch_size=100, shuffle=False)
#print("infered images with labels")
#show_data(infer_dataloader, imagenet_stats, 10, figsize=(20, 8))

### Run batch inference with OpenVINO IR formats

In [ ]:
data_path="data/ThreeClassManualRemove0s/test/"
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(*imagenet_stats)]
)
data = ImageFolderWithPaths(data_path, transform=transform)
dataloader = DataLoader(
    data,
    batch_size=16
)

In [ ]:
import numpy as np
from openvino.runtime import Core, PartialShape

batch_size = 16
model_ir = core.read_model(model=ir_path)
ir_input_layer = model_ir.input(0)
ir_output_layer = model_ir.output(0)
new_shape = PartialShape([batch_size, 3, 224, 224])
model_ir.reshape({ir_input_layer.any_name: new_shape})
config = {"PERFORMANCE_HINT": "THROUGHPUT"}
compiled_model_ir = core.compile_model(model_ir, "BATCH:GPU", config)

imgs = [img for img in dataloader]
score_batch = []
infer_time_sum = 0
start_time = time.time()
for img_infer in imgs:
    
    # For the last batch, the number of input images may be less than previous ones
    # Need to reshape the model for the last batch with smaller batch size
    if img_infer[0].shape[0] != batch_size:
        batch_size = img_infer[0].shape[0]
        new_shape = PartialShape([batch_size, 3, 224, 224])
        model_ir.reshape({ir_input_layer.any_name: new_shape})
        compiled_model_ir = core.compile_model(model_ir, "BATCH:GPU", config)
    
    input_data = img_infer[0].numpy()
    # Batch inference
    start = time.perf_counter()
    output = compiled_model_ir([input_data])
    end = time.perf_counter()
    time_torch = end - start
    
    # Collect the inference results for each batch
    for index in range(batch_size):
        res = list(output.values())[0][index]
        score_batch.append(np.argmax(res))
    
    # Calculate the inference time for all batches
    infer_time_sum += time_torch
    
end_time = time.time()
print(
    f"IR model on GPU for batch inference: {infer_time_sum} total time, "
)
print("Overall running time elapsed for OpenVINO batch inference: ", end_time - start_time)

## Running inference with OpenVINO IR format under AUTO-batching feature

The Automatic-Batching is a new functionality in the OpenVINO™ toolkit. It performs on-the-fly automatic batching (i.e. grouping inference requests together) to improve device utilization, with no programming effort from the user. Therefore, there is no need for the users to create data in batches for inference, while auto-batching will perform batch inference behind the scene automatically.

In [ ]:
def completion_callback(infer_request: InferRequest, image_path: str) -> None:
    
    predictions = next(iter(infer_request.results.values()))
    auto_pred = np.argmax(predictions)


In [ ]:
imgs = []
for x in range(xblock):
    for y in range(yblock):
        filename = 'assets/224/DNM_x{:02d}y{:02d}.jpg'.format(x, y)
        filenameMap = 'assets/224Map_auto/DNM_x{:02d}y{:02d}.png'.format(x, y)
        x_test2 = image_preprocessing(filename)
        imgs.append(x_test2)

In [ ]:
input_tensors = imgs

model_ir = core.read_model(model=ir_path)

# --------------------------- Step 1. Loading model to the device -----------------------------------------------------
compiled_model_async = core.compile_model(model_ir, "BATCH:CPU(16)")

opt_nireq = compiled_model_async.get_property('OPTIMAL_NUMBER_OF_INFER_REQUESTS')
print('OPTIMAL_NUMBER_OF_INFER_REQUESTS', opt_nireq)

config = {"PERFORMANCE_HINT": "THROUGHPUT"}
num_ireq = [4, 256, opt_nireq][1]

# --------------------------- Step 2. Create infer request queue ------------------------------------------------------
# create async queue with optimal number of infer requests
infer_queue = AsyncInferQueue(compiled_model_async, num_ireq)
infer_queue.set_callback(completion_callback)

# --------------------------- Step 3. Do inference --------------------------------------------------------------------
start = time.time()
for i, input_tensor in enumerate(input_tensors):
    infer_queue.start_async({0: input_tensor})
end = time.time()    
infer_queue.wait_all()

print("Inferencing time elapsed for OpenVINO with auto batching: ", end - start)

In [ ]:
# Get inference results from auto-batching inference queue
score_auto = []
for n in range(len(imgs)):
    results = infer_queue[n].get_output_tensor().data
    predArgmax = np.argmax(results)
    score_auto.append(predArgmax)

### Compare the results/accuracy between single image inference and auto-batching

In [ ]:
counter = 0
for k in range(len(score_ir)):
    if int(score_auto[k]) != int(score_ir[k][0]):
        counter += 1
        print('comparing between OpenVINO IR formats with single image inference and auto-batching')
        print("Inference results are different in image", k)
        print(score_auto[k])
        print(score_ir[k][0])
print('counter is', counter)

### Show inference results on images merged into a big map

In [ ]:
def result_to_map(pred, filenameMap):
    """Color each 224 * 224 map with inference results corresponded colors.
       Green for 'Bones highly likely', lightGreen for 'Bones possible', and white for 'No Bones'.
    """
    if pred == '2':
        Image.alpha_composite(img.convert("RGBA"), green).save(filenameMap)
    elif pred == '1': 
        Image.alpha_composite(img.convert("RGBA"), lightGreen).save(filenameMap)
    else:
        Image.alpha_composite(img.convert("RGBA"), black).save(filenameMap)
        
def save_merge_map(merge_path):
    """ Merge the 224 * 224 map into a large map for the final bone finding results.
    """
    map_save = merge_path
    xblock = 17
    yblock = 15
    dst = Image.new('RGB', ((xblock - 1)*224, (yblock - 1)*224))
    for x in range(xblock):
        for y in range(yblock):
            path = 'assets/224Map_auto/DNM_x{:02d}y{:02d}.png'.format(x,y)
            img = Image.open(path)
            dst.paste(img, (x*224, y*224))
            img.close()
    dst.save(map_save)
    print(map_save)
    print("Done!")

In [ ]:
os.makedirs("assets/224Map_auto", exist_ok=True)
xblock = 17
yblock = 15
for x in range(xblock):
    for y in range(yblock):
        filename = 'assets/224/DNM_x{:02d}y{:02d}.jpg'.format(x, y)
        filenameMap = 'assets/224Map_auto/DNM_x{:02d}y{:02d}.png'.format(x, y)
        img = Image.open(filename)
        result_to_map(str(score_auto[x*yblock+y]), filenameMap)

In [ ]:
merge_path = 'assets/DNM_ThreeClassBalanced_openvino_auto.jpg'
save_merge_map(merge_path)

# Run Inference with New Image Tiling Method and New Result Map

## Inference Given Large Image

In [ ]:
import glob
from PIL import Image
from os.path import exists
import os
path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
#path = "data/Explore/Pombal01.PNG"

img = Image.open(path).convert('RGB')
img.size

# Define a Simple Eval Function: single instance

In [ ]:
def transform_image(image):
    my_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    return my_transforms(image).unsqueeze(0)

def eval_simple_torch(working_slice, scratch_model):
    x = transform_image(working_slice)
    output = scratch_model(x)
    return output.detach().numpy().argmax()


### Score large map with PyTorch format in single image inference

In [ ]:
import glob
from os.path import exists
import os
from tqdm import tqdm
#path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
#path = "data/Test/GreenColorado.jpg"
#path = "data/Test/NMMystery1.jpg"
#path = "data/Test/MoabMillCreek.jpg"
img = Image.open(path).convert('RGB')

import time

from PIL import Image
countBuf_torch = np.ones( (img.size[0], img.size[1]) )
sumBuf_torch =   np.zeros( (img.size[0], img.size[1]) )

start = time.time()
step = 16  # choose a factor of 224: 1, 2, 4, 7, 8, 14, 16, 28, 32, 56, 112, and 224. Small is smooth map, large is fast
counts = {0:0, 1:0, 2:0}
scale = {0:0, 1:1, 2:1}
for x in tqdm(range(0, img.size[0]-224, step)):
    for y in range(0, img.size[1]-224, step):  
        bbox = (x, y, x + 224, y + 224)
        working_slice = img.crop(bbox)
        countBuf_torch[bbox[0]:bbox[2], bbox[1]:bbox[3]] += 1
        sumBuf_torch[bbox[0]:bbox[2], bbox[1]:bbox[3]] += scale[eval_simple_torch(working_slice, scratch_model)]
print(f"step size = {step}, Elapsed: {(time.time() - start):6.1f} sec")


In [ ]:
#set opacity to 159: 0 is transparent, 255 is opaque
meanBuf = sumBuf_torch/countBuf_torch
mat = np.uint8(meanBuf.T*159/meanBuf.max()) # scale the opacity: 0 transpartent, 255 solid
idx = mat < 0
mat[idx] = 0
output = Image.fromarray(mat)
output.save('results/bobTile_torch.png')
np.save('results/meanBuf_torch.npy', mat)

In [ ]:
from PIL import ImageDraw
import numpy as np

img = Image.open(path)
alpha = np.load('results/meanBuf_torch.npy')

imgcv = Image.open(path.replace('.jpg','.PNG'))
heatmap_img = np.copy(imgcv)
alpha1D = alpha/alpha.max()*255

heatmap_img[:,:,0] = alpha1D
heatmap_img[:,:,1] = 255 - alpha1D
heatmap_img[:,:,2] = 255 - alpha1D

opacity = .5
imgHeat = Image.blend(Image.fromarray(heatmap_img), imgcv, alpha=0.5)
imgHeat.save('results/HeatMap_torch.png')
print("Color Legend:\n- Red: Higher Probability\n- Blue: -Lower Probability")
imgHeat

### Score large map with OpenVINO IR format in single image inference

In [ ]:
def eval_simple_ir(working_slice, compiled_model_ir, output_layer_ir):
    x = transform_image(working_slice)
    output_ir = compiled_model_ir([x])[output_layer_ir]
    return np.argmax(output_ir)

model_ir = core.read_model(model=ir_path)
compiled_model_ir = core.compile_model(model_ir, "CPU")
output_layer_ir = compiled_model_ir.output(0)
eval_simple_ir(working_slice, compiled_model_ir, output_layer_ir)

In [ ]:
countBuf_ir = np.ones( (img.size[0], img.size[1]) )
sumBuf_ir =   np.zeros( (img.size[0], img.size[1]) )

start = time.time()
step = 16  # choose a factor of 224: 1, 2, 4, 7, 8, 14, 16, 28, 32, 56, 112, and 224. Small is smooth map, large is fast
counts = {0:0, 1:0, 2:0}
scale = {0:0, 1:1, 2:1}
for x in tqdm(range(0, img.size[0]-224, step)):
    for y in range(0, img.size[1]-224, step):  
        bbox = (x, y, x + 224, y + 224)
        working_slice = img.crop(bbox)
        countBuf_ir[bbox[0]:bbox[2], bbox[1]:bbox[3]] += 1
        sumBuf_ir[bbox[0]:bbox[2], bbox[1]:bbox[3]] += scale[eval_simple_ir(working_slice, compiled_model_ir, output_layer_ir)]
print(f"step size = {step}, Elapsed: {(time.time() - start):6.1f} sec")

### Compute the mean of scores for sliding tiles

In [ ]:
#set opacity to 159: 0 is transparent, 255 is opaque
meanBuf_ir = sumBuf_ir/countBuf_ir
mat = np.uint8(meanBuf_ir.T*159/meanBuf_ir.max()) # scale the opacity: 0 transpartent, 255 solid
idx = mat < 0
mat[idx] = 0
output = Image.fromarray(mat)
output.save('results/bobTile.png')
np.save('results/meanBuf.npy', mat)

### Heatmap Approach

Color Legend:
- Bright Red: Bones more likely
- Bright Blue: Bone not likely

In [ ]:
from PIL import ImageDraw
import numpy as np

img = Image.open(path)
alpha = np.load('results/meanBuf.npy')

imgcv = Image.open(path.replace('.jpg','.PNG'))
heatmap_img = np.copy(imgcv)
alpha1D = alpha/alpha.max()*255

heatmap_img[:,:,0] = alpha1D
heatmap_img[:,:,1] = 255 - alpha1D
heatmap_img[:,:,2] = 255 - alpha1D

opacity = .5
imgHeat = Image.blend(Image.fromarray(heatmap_img), imgcv, alpha=0.5)
imgHeat.save('results/HeatMap.png')
print("Color Legend:\n- Red: Higher Probability\n- Blue: -Lower Probability")
imgHeat

### Score large map with OpenVINO IR format and AUTO-batching

In [ ]:
img = Image.open(path).convert('RGB')

imgs = []
countBuf_auto = np.ones( (img.size[0], img.size[1]) )
sumBuf_auto =   np.zeros( (img.size[0], img.size[1]) )

start = time.time()
step = 16  # choose a factor of 224: 1, 2, 4, 7, 8, 14, 16, 28, 32, 56, 112, and 224. Small is smooth map, large is fast
counts = {0:0, 1:0, 2:0}
scale = {0:0, 1:1, 2:1}
for x in tqdm(range(0, img.size[0]-224, step)):
    for y in range(0, img.size[1]-224, step):  
        bbox = (x, y, x + 224, y + 224)
        working_slice = img.crop(bbox)
        imgs.append(transform_image(working_slice))

In [ ]:
len(imgs)

In [ ]:
model_ir = core.read_model(model=ir_path)
score_auto_new = []

compiled_model_async = core.compile_model(model_ir, "BATCH:CPU(16)")

opt_nireq = compiled_model_async.get_property('OPTIMAL_NUMBER_OF_INFER_REQUESTS')

config = {"PERFORMANCE_HINT": "THROUGHPUT"}
num_ireq = [4, 256, opt_nireq][1]

# bn is the number of iterations of auto-batching inference
# since the number of input images is too large (>50,000), need to split them according to "OPTIMAL_NUMBER_OF_INFER_REQUESTS"
bn = len(range(0, len(imgs), num_ireq))
print("TOTAL NUMBER OF ITERATIONS FOR AUTO BATCHING IS ", bn)

# --------------------- Set up the inference queue for auto-batching and do inference ------------------------------------------
start = time.time()
infer_queue = AsyncInferQueue(compiled_model_async, num_ireq)
for batch in range(0, bn): 

    infer_queue.set_callback(completion_callback)
    
    if (num_ireq+batch*num_ireq) < len(imgs):
        input_tensors = imgs[(0+batch*num_ireq):(num_ireq+batch*num_ireq)]
    else:
        input_tensors = imgs[(0+batch*num_ireq):]
        
    for i, input_tensor in enumerate(input_tensors):
        infer_queue.start_async({0: input_tensor})
    
    infer_queue.wait_all()
    
    for n in range(len(input_tensors)):
        result = infer_queue[n].get_output_tensor().data
        pred = np.argmax(result)
        score_auto_new.append(pred)
end = time.time()

print("Inferencing time elapsed for OpenVINO auto batching: ", end - start)

In [ ]:
# Post-processing for inference results from iterations of auto-batching

xblock = len(range(0, img.size[0]-224, step))
yblock = len(range(0, img.size[1]-224, step))
xindex = 0
yindex = 0
countBuf_auto = np.ones( (img.size[0], img.size[1]) )
sumBuf_auto =   np.zeros( (img.size[0], img.size[1]) )
counts = {0:0, 1:0, 2:0}
scale = {0:0, 1:1, 2:1}

for x in tqdm(range(0, img.size[0]-224, step)):
    for y in range(0, img.size[1]-224, step):
        bbox = (x, y, x + 224, y + 224)
        pred = score_auto_new[xindex*yblock+yindex]
        countBuf_auto[bbox[0]:bbox[2], bbox[1]:bbox[3]] += 1
        sumBuf_auto[bbox[0]:bbox[2], bbox[1]:bbox[3]] += scale[pred]
        yindex += 1
    xindex += 1
    yindex = 0

In [ ]:
img.size[0]-224

In [ ]:
#set opacity to 159: 0 is transparent, 255 is opaque
meanBuf_auto = sumBuf_auto/countBuf_auto
mat = np.uint8(meanBuf_auto.T*159/meanBuf_auto.max()) # scale the opacity: 0 transpartent, 255 solid
idx = mat < 0
mat[idx] = 0
output = Image.fromarray(mat)
output.save('results/bobTile_auto.png')
np.save('results/meanBuf_auto.npy', mat)

In [ ]:
img = Image.open(path)
alpha = np.load('results/meanBuf_auto.npy')

imgcv = Image.open(path.replace('.jpg','.PNG'))
heatmap_img = np.copy(imgcv)
alpha1D = alpha/alpha.max()*255

heatmap_img[:,:,0] = alpha1D
heatmap_img[:,:,1] = 255 - alpha1D
heatmap_img[:,:,2] = 255 - alpha1D

opacity = .5
imgHeat = Image.blend(Image.fromarray(heatmap_img), imgcv, alpha=0.5)
imgHeat.save('results/HeatMap_auto.png')
print("Color Legend:\n- Red: Higher Probability\n- Blue: -Lower Probability")
imgHeat

## To learn more about OpenVINO. Try the demo link below.
* https://github.com/openvinotoolkit/openvino_notebooks